In [1]:
import requests
import zipfile
import os
from tqdm import tqdm
import csv
from collections import defaultdict
import pandas as pd
url = "https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip"
local_filename = "MINDsmall_train.zip"
def download_file(url, local_filename):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024

    with open(local_filename, "wb") as file:
        for data in tqdm(response.iter_content(block_size), total=total_size // block_size, unit='KiB', unit_scale=True):
            file.write(data)
    print("File downloaded successfully.")
def unzip_file(zip_filename, extract_to):
    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
        total_files = len(zip_ref.infolist())
        for file in tqdm(zip_ref.infolist(), total=total_files, unit='file'):
            zip_ref.extract(file, extract_to)
    print("File unzipped successfully.")
download_file(url, local_filename)
unzip_file(local_filename, "database/MINDsmall_train")
os.remove(local_filename)
print("Zip file removed.")
file_path = 'database/MINDsmall_train/behaviors.tsv'
output_file = 'database/topNews.csv'
recommended_counts = defaultdict(int)
with open(file_path, 'r', encoding='utf-8') as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter='\t')
    for line in tsvreader:
        if len(line) >= 4:
            recommended_articles = line[3].split()
            for article in recommended_articles:
                recommended_counts[article] += 1
top_10_recommended = sorted(recommended_counts.items(), key=lambda x: x[1], reverse=True)[:10]
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Article ID', 'Count'])
    for article_id, count in top_10_recommended:
        csvwriter.writerow([article_id, count])
print(f"Top 10 recommended news articles saved to {output_file}")
df = pd.read_csv('database/MINDsmall_train/behaviors.tsv',sep='\t', header=None)
df.columns = ['ID', 'User', 'Timestamp', 'NewsIDs', 'Interactions']
like_count = {}
dislike_count = {}
neutral_count = {}
total_clicks = {}
for index, row in df.iterrows():
    if isinstance(row['NewsIDs'], str) and isinstance(row['Interactions'], str):
        news_ids = row['NewsIDs'].split()
        interactions = row['Interactions'].split()
        for news_id in news_ids:
            if news_id not in like_count:
                like_count[news_id] = 0
                dislike_count[news_id] = 0
                neutral_count[news_id] = 0
                total_clicks[news_id] = 0
            neutral_count[news_id] += 1
            total_clicks[news_id] += 1
        for interaction in interactions:
            news_id, value = interaction.rsplit('-', 1)
            if news_id not in like_count:
                like_count[news_id] = 0
                dislike_count[news_id] = 0
                neutral_count[news_id] = 0
                total_clicks[news_id] = 0
            if value == '1':
                like_count[news_id] += 1
            elif value == '0':
                dislike_count[news_id] += 1
            total_clicks[news_id] += 1
result_df = pd.DataFrame({
    'NewsID': list(like_count.keys()),
    'Likes': list(like_count.values()),
    'Dislikes': list(dislike_count.values()),
    'Neutrals': list(neutral_count.values()),
    'Total Clicks': list(total_clicks.values())
})
result_df.to_csv('database/newsData.csv', index=False)
print("Data saved to newsData.csv")
df = pd.read_csv('database/MINDsmall_train/behaviors.tsv',sep='\t', header=None)
df.to_csv('database/behaviors.csv', index=False, header=None)
df = pd.read_csv('database/MINDsmall_train/news.tsv',sep='\t', header=None)
df.to_csv('database/news.csv', index=False, header=None)
print("Data saved to news.csv")
id_list = []
with open('database/MINDsmall_train/behaviors.tsv', 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        if len(row) > 1:
            if row[1] not in id_list:
                id_list.append({'id': row[0],'pass':row[0]})
idx = pd.DataFrame(id_list)
idx.to_csv('database/userpass.csv', index=False)

51.7kKiB [14:17, 60.3KiB/s]                          


File downloaded successfully.


100%|██████████| 4/4 [00:00<00:00,  7.14file/s]


File unzipped successfully.
Zip file removed.
Top 10 recommended news articles saved to database/topNews.csv
Data saved to newsData.csv
Data saved to news.csv


TypeError: argument of type 'builtin_function_or_method' is not iterable